In [1]:
"""These are the tools we will use to play with the data"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
"""This is the data we will be playing with"""
import CPTAC

Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptome Data...
Loading CNA Data...
Loading Phosphoproteomics Data...


In [3]:
"""Use case 3: BMI above and below 25"""
"""The first step is to load the clinical dataframe and the dataframe to compare it with"""
clinical = CPTAC.get_clinical()
cna = CPTAC.get_CNA() #TODO: this will eventually be proteomics once we figure out how to meld indices

"""Next we will use the compare_clinical() function to create a dataframe that appends a column from the clinical dataframe to our chosen dataframe"""
bmiCNA = CPTAC.compare_clinical(clinical, cna, 'BMI')
print(bmiCNA)

AttributeError: module 'CPTAC' has no attribute 'get_clinical'

In [ ]:
"""Now we can split the bmiCNA dataframe into two dataframes based on whether the patient has a BMI (or whatever variable was specified in the previous step) above or below 25.
This is done by using the dataframe logic to create an array of boolean values, which we can then use to select the respective dataframes"""
greaterBool = bmiCNA['BMI'] >= 25
lessBool = bmiCNA['BMI'] < 25

largerThan25 = bmiCNA[greaterBool]
smallerThan25 = bmiCNA[lessBool]
print(largerThan25)
print(smallerThan25)

In [ ]:
"""We can now apply the mean() function to these two dataframes to produce an array of mean values for each column in our dataframe (e.g. each gene)"""
largerMean = largerThan25.mean()
smallerMean = smallerThan25.mean()
print(largerMean)
print(smallerMean)

In [ ]:
"""Now we subtract one dataframe from the other and take the absolute value"""
difference = abs(largerMean - smallerMean)

"""We should get ride of the BMI value, as a mean difference for the BMI has no meaning here (this can be done in previous steps with both dataframes if you prefer)"""
difference = difference[1:]
print(difference)

In [ ]:
"""The final step is to create a histogram of our mean difference values"""
#TODO do this through matplotlib, seaborn?